<h1>ATMOS Version 1.0.0</h1>
The ATMOS calculator, written in Python. It is able to: 
<li>Determine atmos properties for any given altitude</li>
<li>Determine all airspeeds (including Mach) given a speed and altitude</li>



In [192]:
# Python Imports
import pandas as pd
import numpy as np
import math
import csv

<h2>Load the Standard Atmosphere</h2>
Imports standard atmosphere from a file "/instance/standard_atmos.csv".

Parameters are:
<li>Hg:       Geometric Altitude [ft]</li>
<li>H:        Geopotential Altitude [ft]</li>
<li>T:        Temperature [deg_R]</li>
<li>P:        Presure [lb/ft3]</li>
<li>P/P0:     Presure Ratio </li>
<li>pho:      Density [slug/ft3] </li>
<li>pho/pho0: Density Ratio </li>

In [193]:
# Open csv
df_atmos = pd.read_csv('./instance/standard_atmos.csv', comment="#")

print(df_atmos.head(3))


   Hgeo_ft    H_ft   T_degR  P_lb/ft3    P/P0  pho_slug/ft3  pho/pho0
0    -2500 -2500.0  527.584    2300.5  1.0871      0.002547    1.0711
1        0     0.0  518.670    2116.2  1.0000      0.002378    1.0000
2     2500  2500.0  509.756    1931.9  0.9129      0.002209    0.9289


<h2>User Defines the condition to analize</h2>
User selects
<li>Altitude pressure in ft</li>
<li>Speed KEAS, KCAS or KTAS</li>
<li>Speed value in Kts</li>

In [194]:
h_press_user = 10000 # Units ft
analysis_type = "KCAS" # "KEAS", "KCAS" or "KTAS" acceptable
user_speed = 86.036    # Units kts

<h2>Calculation of Atmosheric Parameters</h2>
Process flow:
<li>Extract sealevel contants</li>
<li>Test request is "on table"</li>
<li>Interpolate atmos properties to requested altitude</li>
<li>Caculate airspeeds and Mach</li>

In [195]:
# Extract P at sealevel
p_0 = df_atmos.loc[df_atmos["Hgeo_ft"]==0, "P_lb/ft3"].values[0] 
print(p_0)

2116.2


In [196]:
# Test request is "on table"
hgeo_min = df_atmos["Hgeo_ft"].min()
hgeo_max = df_atmos["Hgeo_ft"].max()

# Test user hgeo winthin table
if hgeo_min > h_press_user:
  print("ERROR: Altitude too low out of range")
  exit()
elif hgeo_max < h_press_user:
  print("ERROR: Altitude too high out of range")
  exit()

In [197]:
# Interpolate atmos properties to requested altitude
pTarget        = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["P_lb/ft3"])
phoRatioTarget = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["pho/pho0"])
phoTarget      = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["pho_slug/ft3"])

print(f"For a requested altitude of: {h_press_user} ft")
print(f"Pressure [lb/ft2]: {pTarget}")
print(f"Density [slug/ft3]: {phoTarget}")


For a requested altitude of: 10000 ft
Pressure [lb/ft2]: 1455.6
Density [slug/ft3]: 0.001756124


In [198]:
# Calculate Airspeed

def speed_calc (analysis, speed_defined, phoRatioTarget, pTarget, p_0):
  if analysis == "KEAS":  # Method: ALT AND KEAS
    KEASTarget = speed_defined
    aTarget    = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    KTASTarget = KEASTarget/math.sqrt(phoRatioTarget)
    MachTarget = KTASTarget/aTarget
    qTarget    = pTarget*((1+0.2*MachTarget**2)**3.5-1)
    KCASTarget = 661.47*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis == "KTAS":  # Method ALT and KTAS
    KTASTarget = speed_defined
    aTarget    = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    KEASTarget = KTASTarget*math.sqrt(phoRatioTarget)
    MachTarget = KTASTarget/aTarget
    qTarget    = pTarget*((1+0.2*MachTarget**2)**3.5-1)
    KCASTarget = 661.47*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis == "KCAS":  # Method ALT and KCAS    
    KCASTarget = speed_defined
    aTarget = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    qTarget =  p_0*((0.2*(KCASTarget/661.47)**2+1)**3.5-1)
    MachTarget = math.sqrt(5*((qTarget/pTarget+1)**(2/7)-1))
    KTASTarget = aTarget*MachTarget
    KEASTarget = KTASTarget*math.sqrt(phoRatioTarget)
  else:
    print("Requested analysis not available")
    KTASTarget = np.NaN
    KCASTarget = np.NaN
    KEASTarget = np.NaN
    MachTarget = np.NaN
    qTarget = np.NaN

  df = {
    'KCAS': KCASTarget,
    'KEAS': KEASTarget,
    'KTAS': KTASTarget,
    'Mach': MachTarget,
    'q': qTarget,
  }

  return df

output = speed_calc(
  analysis=analysis_type,
  speed_defined=user_speed,
  phoRatioTarget=phoRatioTarget,
  pTarget=pTarget,
  p_0=p_0,
  )

print(output)

{'KCAS': 86.036, 'KEAS': 85.94134449558962, 'KTAS': 99.99942251193673, 'Mach': 0.1566799988406297, 'q': 25.166968214451412}


<h2>Output</h2>
Prints output to user and appends to log file.

In [199]:


with open('AtmosLog.txt', 'w') as f:  
    w = csv.DictWriter(f, output.keys())
    w.writeheader()
    w.writerow(output)
